In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[ 0.0562, -0.0742,  0.0936,  ..., -0.0549, -0.1597, -0.0070],
        [ 0.0556,  0.0978, -0.0003,  ..., -0.0715, -0.1275, -0.1347],
        [ 0.0395, -0.0373,  0.0753,  ..., -0.0290, -0.0486, -0.0680],
        ...,
        [ 0.0982,  0.0166,  0.0759,  ..., -0.0721, -0.0387, -0.0974],
        [ 0.1201, -0.1161, -0.1194,  ..., -0.1480, -0.0802, -0.0264],
        [ 0.0351, -0.1009,  0.0264,  ..., -0.0326, -0.0093, -0.1119]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-19.4352, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.0513, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.4483, device='cuda:0', grad_fn=<NegBackward0>), tensor(-50.1894, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-10.8564, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.4374, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.0397, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3528, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-10.8564, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.4374, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.0397, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3528, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-21.9319, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.2069, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.7195, device='cuda:0', grad_fn=<NegBackward0>), tensor(-53.8949, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-21.9319, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-42.5615, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.4822, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.7545, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.5480, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-42.5615, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.4822, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.7545, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.5480, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-43.3815, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.8943, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.1591, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.8018, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-43.3815, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.8943, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.1591, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.8018, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-43.7019, device='cuda:0', grad_fn=<NegBackward0>

tensor(-50.8239, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.4916, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.9505, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.9826, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-50.2755, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.3158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.5882, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.1679, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-50.2755, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.3158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.5882, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.1678, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-51.2786, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.7309, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.1661, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.9730, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-51.2787, device='cuda:0', grad_fn=<NegBackward0>

tensor(-56.3155, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.1280, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.7642, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.3708, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-56.3155, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.1280, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.7642, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.3709, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-57.0799, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.5065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.1479, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.5596, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-57.0799, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.5065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.1479, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.5596, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-57.6204, device='cuda:0', grad_fn=<NegBackward0>

tensor(-63.1330, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0584, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.5991, device='cuda:0', grad_fn=<NegBackward0>), tensor(-86.8433, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-62.6452, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.9681, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.2066, device='cuda:0', grad_fn=<NegBackward0>), tensor(-86.3957, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-62.6452, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.9681, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.2066, device='cuda:0', grad_fn=<NegBackward0>), tensor(-86.3957, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-62.3325, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.9624, device='cuda:0', grad_fn=<NegBackward0>), tensor(-26.9047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-85.9494, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-62.3325, device='cuda:0', grad_fn=<NegBackward0>

...

tensor(-98.5600, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.1207, device='cuda:0', grad_fn=<NegBackward0>), tensor(-46.7751, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.0858, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-98.8778, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.1881, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.0246, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.1737, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-98.8778, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.1881, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.0246, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.1737, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-98.9414, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.2405, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.0350, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.2587, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-98.9414, device='cuda:0', grad_fn=<NegBackwa

tensor(-94.7092, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.7085, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.3568, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.9545, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-94.7092, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.7085, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.3568, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.9545, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.7768, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-46.1255, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.0010, device='cuda:0', grad_fn=<NegBackward0>), tensor(-103.6194, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.7768, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-46.1255, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.0010, device='cuda:0', grad_fn=<NegBackward0>), tensor(-103.6194, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.7799, device='cuda:0', grad_fn=<NegBackwa

tensor(-99.6319, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5775, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.3867, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.4433, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.9329, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6770, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5862, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.6675, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.9329, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6770, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5862, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.6675, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.8429, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6838, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4903, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.5766, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.8429, device='cuda:0', grad_fn=<NegBackwa

tensor(-94.3815, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.5386, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.1986, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.9863, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-94.3815, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.5386, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.1986, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.9863, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-84.7282, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-40.6465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.4860, device='cuda:0', grad_fn=<NegBackward0>), tensor(-98.0528, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-84.7282, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-40.6465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.4860, device='cuda:0', grad_fn=<NegBackward0>), tensor(-98.0528, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-85.1749, device='cuda:0', grad_fn=<NegBackward

tensor(-96.6960, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-46.5960, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.4468, device='cuda:0', grad_fn=<NegBackward0>), tensor(-103.9212, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.8510, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-46.6552, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.5412, device='cuda:0', grad_fn=<NegBackward0>), tensor(-104.0614, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.8510, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-46.6552, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.5412, device='cuda:0', grad_fn=<NegBackward0>), tensor(-104.0614, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.6194, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-46.5604, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.4061, device='cuda:0', grad_fn=<NegBackward0>), tensor(-103.8927, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.6194, device='cuda:0', grad_fn=<NegBackwa

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.04it/s]


----------------------------
Train_z, Test_z
PCA: 0.9608
SVD: 0.9591
NetD, Acc at first 0.9682
